# Scraping

Obtener todos los datos a base de Scrapeo:

### Fases
- Con Selenium, obtener todas las urls de los supermercados
- Con Selenium, utilizar las URLs obtenidas, para sacar las urls de:
    - Aceite girasol
    - Aceite de oliva
    - Leche
- Con selenium, sacar las URLs de las subcategorias de:
    - Aceite de oliva
        - Suave e intenso
        - Virgen
        - Virgen extra.
    - Leche
        - Enriquecida
        - Entera, semi o desnatada
        - Sin Lactosa
    De forma que, por supermercado, nos quedaremos con:
    - URL Aceite Girasol
    - URL Aceite de Oliva Suave e Intenso
    - URL Aceite de Oliva Virgen
    - URL Aceite de Oliva Virgen extra
    - URL leche enriquecida
    - URL leche entera,semi o desnatada
    - URL leche sin lactosa

Con estas URL utilizaremos BEAUTIFUL SOUP para sacar las URL del histórico de cada producto

In [23]:
# Importamos las librerías que necesitamos

# Librerías para automatización de navegadores (Selenium)
# -----------------------------------------------------------------------
from selenium import webdriver                   # Control automático del navegador para realizar scraping web

from selenium.webdriver.chrome.service import Service  # Maneja el servicio de ChromeDriver
from selenium.webdriver.common.by import By      # Para seleccionar elementos en la página web (DOM)
from selenium.webdriver.chrome.options import Options  # Configuración de opciones del navegador (headless mode, etc.)
import time                                      # Para gestionar pausas y temporización

# Librerías para extracción de datos y scraping
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup  # Para analizar y extraer datos de HTML y XML (scraping web)

# Librerías para tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd             # Para manipulación de estructuras de datos como DataFrames
import numpy as np              # Para cálculos numéricos y manejo de arrays
import datetime                 # Para manipulación de fechas y tiempos
# -----------------------------------------------------------------------
# Librerías para visualización de progreso
# -----------------------------------------------------------------------
from tqdm import tqdm  # Para crear barras de progreso durante los bucles

# Manejo de tiempo y solicitudes con espera
# -----------------------------------------------------------------------
from time import sleep          # Para pausar la ejecución del código por un tiempo determinado
import random                   # Para generar valores aleatorios, útil para espaciar solicitudes y evitar bloqueos
# Importación de módulos desde rutas recursivas
# -----------------------------------------------------------------------
import sys
sys.path.append("../")  # Añadir rutas externas para poder importar módulos desde carpetas superiores
from src.SupportScraping import obtener_url_supermercados,obtener_url_productos_por_supermercado,obtener_urls_por_super_producto_y_categoria,obtener_href_productos,crear_df_productos_con_historico# Funciones personalizadas para obtener datos de vuelos

### Obtener con Selenium, obtener todas las urls de los supermercados

In [24]:
xpaths = ["/html/body/section[1]/div/div[2]/div[1]/div/div[2]",
          "/html/body/section[1]/div/div[2]/div[2]/div/div[2]/div",
          "/html/body/section[1]/div/div[2]/div[3]/div/div[2]/div",
          "/html/body/section[1]/div/div[2]/div[4]/div/div[2]/div",
          "/html/body/section[1]/div/div[2]/div[5]/div/div[2]/div",
          "/html/body/section[1]/div/div[2]/div[6]/div/div[2]/div"]

url_supermercados = [obtener_url_supermercados(xpath) for xpath in tqdm(xpaths)]

100%|██████████| 6/6 [01:12<00:00, 12.06s/it]


Observamos el resultado

In [25]:
url_supermercados

['https://super.facua.org/mercadona/',
 'https://super.facua.org/carrefour/',
 'https://super.facua.org/eroski/',
 'https://super.facua.org/dia/',
 'https://super.facua.org/hipercor/',
 'https://super.facua.org/alcampo/']

## Con Selenium, utilizar las URLs obtenidas, para sacar las urls de:
- Aceite girasol
- Aceite de oliva
- Leche

In [26]:
url_productos = [obtener_url_productos_por_supermercado(url) for url in tqdm(url_supermercados)]

100%|██████████| 6/6 [05:14<00:00, 52.36s/it]


In [27]:
url_productos

[['https://super.facua.org/mercadona/aceite-de-girasol/',
  'https://super.facua.org/mercadona/aceite-de-oliva/',
  'https://super.facua.org/mercadona/leche/'],
 ['https://super.facua.org/carrefour/aceite-de-girasol/',
  'https://super.facua.org/carrefour/aceite-de-oliva/',
  'https://super.facua.org/carrefour/leche/'],
 ['https://super.facua.org/eroski/aceite-de-girasol/',
  'https://super.facua.org/eroski/aceite-de-oliva/',
  'https://super.facua.org/eroski/leche/'],
 ['https://super.facua.org/dia/aceite-de-girasol/',
  'https://super.facua.org/dia/aceite-de-oliva/',
  'https://super.facua.org/dia/leche/'],
 ['https://super.facua.org/hipercor/aceite-de-girasol/',
  'https://super.facua.org/hipercor/aceite-de-oliva/',
  'https://super.facua.org/hipercor/leche/'],
 ['https://super.facua.org/alcampo/aceite-de-girasol/',
  'https://super.facua.org/alcampo/aceite-de-oliva/',
  'https://super.facua.org/alcampo/leche/']]

In [33]:
urls_super_category = [obtener_urls_por_super_producto_y_categoria(url_productos[i][0],url_productos[i][1],url_productos[i][2]) for i,p in enumerate(url_productos)]

### Obtener las URL para acceder a las tablas de historico

In [34]:
supermercados_dfs = [obtener_href_productos(df) for df in tqdm(urls_super_category)]



0it [09:07, ?it/s]
0it [07:13, ?it/s]
  0%|          | 0/6 [07:13<?, ?it/s]





100%|██████████| 6/6 [00:17<00:00,  3.00s/it]


### Sacamos la última tabla de la operación anterior que nos unifica todo en una

In [35]:
df = pd.DataFrame()
lista_dfs = []
for i in range(0,len(supermercados_dfs)):
    for e in range(0,len(supermercados_dfs[i])):
        df_temp = supermercados_dfs[i][e]
        df = pd.concat([df,df_temp])
    
    lista_dfs.append(df)

In [36]:
df = lista_dfs[5]

In [37]:
df.sample(6)

,supermercado,categoria,Producto,URL
49,hipercor,leche-entera-semi-desnatada,El Corte Ingles Leche Desnatada De Asturias Br...,https://super.facua.org/hipercor/leche/el-cort...
72,alcampo,leche-entera-semi-desnatada,Möntbelle Leche Fresca De Cabra 750 Ml.,https://super.facua.org/alcampo/leche/mntbelle...
13,hipercor,leche-enriquecida,Gaza Leche Semidesnatada Con Calcio Brik 1 L,https://super.facua.org/hipercor/leche/gaza-le...
1,mercadona,leche-sin-lactosa,Leche Desnatada Sin Lactosa Hacendado 6 L.,https://super.facua.org/mercadona/leche/leche-...
1,mercadona,aceite-de-oliva-suave-e-intenso,Aceite De Oliva Suave Hacendado 3 L.,https://super.facua.org/mercadona/aceite-de-ol...
10,hipercor,aceite-de-oliva-suave-e-intenso,"Carbonell Aceite De Oliva Suave 0,4º Botella 1 L",https://super.facua.org/hipercor/aceite-de-oli...


In [38]:
df.reset_index(drop=True,inplace=True)

In [39]:
df.to_csv("../datos/backups/URLS_historico_productos.csv")

In [40]:
df = pd.read_csv("../datos/backups/URLS_historico_productos.csv",index_col="Unnamed: 0")

In [42]:
df.head()

,supermercado,categoria,Producto,URL
0,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",https://super.facua.org/mercadona/aceite-de-gi...
1,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 5 L.",https://super.facua.org/mercadona/aceite-de-gi...
2,mercadona,aceite-de-oliva-suave-e-intenso,Aceite De Oliva Intenso Hacendado 3 L.,https://super.facua.org/mercadona/aceite-de-ol...
3,mercadona,aceite-de-oliva-suave-e-intenso,Aceite De Oliva Suave Hacendado 3 L.,https://super.facua.org/mercadona/aceite-de-ol...
4,mercadona,aceite-de-oliva-virgen,Aceite De Oliva Virgen Hacendado 1 L.,https://super.facua.org/mercadona/aceite-de-ol...


In [4]:
tablas_historicos = crear_df_productos_con_historico(df)

In [11]:
len(tablas_historicos)

1531

In [14]:
df_historicos = pd.concat(tablas_historicos)

In [15]:
df_historicos

,supermercado,categoria,producto,Día,Precio (€),Variación
0,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",12/07/2024,1.45,=
1,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",13/07/2024,1.45,=
2,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",14/07/2024,1.45,=
3,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",15/07/2024,1.45,=
4,mercadona,aceite-de-girasol-,"Aceite De Girasol Refinado 0,2º Hacendado 1 L.",16/07/2024,1.45,=
...,...,...,...,...,...,...
96,alcampo,leche-sin-lactosa,Puleva Proteína Extra Pro Leche De Vaca Desnat...,21/10/2024,7.74,"+0,24 (3,20%)"
97,alcampo,leche-sin-lactosa,Puleva Proteína Extra Pro Leche De Vaca Desnat...,22/10/2024,7.74,=
98,alcampo,leche-sin-lactosa,Puleva Proteína Extra Pro Leche De Vaca Desnat...,23/10/2024,7.74,=
99,alcampo,leche-sin-lactosa,Puleva Proteína Extra Pro Leche De Vaca Desnat...,24/10/2024,7.74,=
